In [1]:
import json
with open('./data/CVPR_2022_NAS_Track2_train.json', 'r') as f:
    train_data = json.load(f)

In [2]:
import pandas as pd
import numpy as np

In [3]:
target_cols = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']

In [4]:
def get_df(train_data):
    ret = []
    for k, v in train_data.items():
        tmp = list(v['arch'])
        tmp1 = []
        for c in target_cols:
            tmp1.append(v[c])
        ret.append(tmp+tmp1+[k,v['arch']])
    retf = pd.DataFrame(ret,columns=[f'col{_}' for _ in range(len(tmp))]+target_cols+['id','arch'])
    retf['col0'] = retf['col0'].map({'l':1, 'j':2, 'k':3})
    int_cols = [x for x in retf.columns if x not in ['id','arch']]
    retf[int_cols] = retf[int_cols].astype(float)
    return retf

In [5]:
train_df = get_df(train_data)

In [6]:
from scipy.special import erfinv 

for c in target_cols:
    train_y=train_df[c]
    mmin=np.min(train_y)+1
    mmax=np.max(train_y)+1
    train_y=np.sqrt(2) * erfinv(2 * (train_y+mmin)/(mmin+mmax)-1)
    train_df[c+'_trans_y'] = train_y

In [7]:
def get_step(train_df):
    res0 = []
    res1 = []
    res2 = []
    res3 = []
    res4 = []
    map_={'l':1, 'j':2, 'k':3}
    time_step=12
    head_ = set([1+_*3 for _ in range(12)])
    mlp_ = set([2+_*3 for _ in range(12)])
    emb_ = set([3+_*3 for _ in range(12)])
    depth_ = set([0])
    for item in train_df.arch:
        ret = np.array(list(item[1:]),dtype=np.float32).reshape(-1,3)
        res0.append([1 if x in head_ else 0 for x in range(37)])
        res1.append([1 if x in mlp_ else 0 for x in range(37)])
        res2.append([1 if x in emb_ else 0 for x in range(37)])
        res3.append([1 if x in depth_ else 0 for x in range(37)])
        res4.append([map_[item[0]]]+list(np.array(list(item[1:]),dtype=np.float32)))
        
    train_df['head'] = res0
    train_df['mlp'] = res1
    train_df['emb'] = res2
    train_df['depth'] = res3
    train_df['all_emb'] = res4

In [8]:
get_step(train_df)

In [9]:
from sklearn.model_selection import KFold
from tqdm import tqdm

In [10]:
#定义数据
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self,df,use_cols,target_cols,show=0):
        self.df = df
        self.show = show
        self.use_cols = use_cols
        self.target_cols = target_cols

        self.prepare_data()
        
    def __len__(self):
        return self.df.shape[0]
    
    def prepare_data(self):
        self.y = self.df[self.target_cols].values
        self.y0 = self.df[[self.target_cols[0]]].values
        self.y1 = self.df[[self.target_cols[1]]].values
        self.y2 = self.df[[self.target_cols[2]]].values
        self.y3 = self.df[[self.target_cols[3]]].values
        self.y4 = self.df[[self.target_cols[4]]].values
        self.y5 = self.df[[self.target_cols[5]]].values
        self.y6 = self.df[[self.target_cols[6]]].values
        self.y7 = self.df[[self.target_cols[7]]].values
        
        uc = ['all_emb']
        tmp_dt = {}
        for c in uc:
            tmp_dt[c] = np.array(self.df[c].tolist())
        self.inputs = np.concatenate([tmp_dt[c][:, None] for c in uc], 1).transpose(0, 2, 1)

        uc= ['all_emb','depth','head','mlp','emb']
        tmp_dt = {}
        for c in uc:
            tmp_dt[c] = np.array(self.df[c].tolist())
        self.inputs1= np.concatenate([tmp_dt[c][:, None] for c in uc], 1).transpose(0, 2, 1)
        
        
        if self.show==1:
            print('inputs_shape',self.inputs.shape)
            print('inputs1_shape',self.inputs1.shape)
            print('y_shape',self.y.shape)
        
    def __getitem__(self, idx):
        data = {
            "input": torch.tensor(self.inputs[idx], dtype=torch.float),
            "input1": torch.tensor(self.inputs1[idx], dtype=torch.float),
            "y": torch.tensor(self.y[idx], dtype=torch.float),
            "y0": torch.tensor(self.y0[idx], dtype=torch.float),
            "y1": torch.tensor(self.y1[idx], dtype=torch.float),
            "y2": torch.tensor(self.y2[idx], dtype=torch.float),
            "y3": torch.tensor(self.y3[idx], dtype=torch.float),
            "y4": torch.tensor(self.y4[idx], dtype=torch.float),
            "y5": torch.tensor(self.y5[idx], dtype=torch.float),
            "y6": torch.tensor(self.y6[idx], dtype=torch.float),
            "y7": torch.tensor(self.y7[idx], dtype=torch.float),
        }
        
        return data

In [11]:
use_cols = [x for x in train_df.columns if 'col' in x]

In [12]:
len(use_cols)

37

In [13]:
import gc
import os
import time
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup


def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    
def count_parameters(model, all=False):
    if all:
        return sum(p.numel() for p in model.parameters())
    else:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)


def save_model_weights(model, modelpath, filename):
    torch.save(model.state_dict(), modelpath+filename)
    return f"\n -> Save weights to {modelpath+filename}\n"

In [14]:
import scipy

In [15]:
def compute_metric(pred, y):
    corr = []
    if pred.shape[1]>2:
        for i in range(8):
            corr.append(scipy.stats.stats.kendalltau(pred[:, i], y[:, i])[0])
    else:
        corr.append(scipy.stats.stats.kendalltau(pred, y)[0])
    return np.array(corr)
    
class CVPRLoss_tanh(nn.Module):
    # kendall tanh
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([((tmp[i:]-tmp[i-1])).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1


In [16]:
class CVPRModel(nn.Module):
    def __init__(
        self,
        input_dim=3,
        num_classes=8,
        time_step=12,
        bi=True
    ):
        super(CVPRModel,self).__init__()
        
        self.bi_num = 2 if bi else 1
        self.time_step = time_step
        
        self.MLP = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
        )
        
        self.LSTM0 = nn.LSTM(256, 256, 
                    batch_first=True, 
                    bidirectional=bi, 
                    num_layers=1,
                    dropout=0)
                                 
        
        self.LSTM1 = nn.LSTM(512, 256, 
                    batch_first=True, 
                    bidirectional=bi, 
                    num_layers=1,
                    dropout=0
                            )    

        self.Logits = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear((512+512+256+1)*self.time_step, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        x1 = self.MLP(x)
        x2, (h0, c0) = self.LSTM0(x1)
        x3, (h1, c1) = self.LSTM1(x2, (h0, c0))

        c1 = torch.cat([x3, x2, x1, x], -1)
        
        pred = self.Logits(c1)
        return pred

In [17]:
def train_sig(model, 
        train_dataset, 
        val_dataset, 
        verbose=20, 
        fold_=0,
        modelname='MLP_base',
        modelpath=r'./model'+'//',
        input='input',
        y='y',
        early_stop_round=60,
        debug=False):
    
    print(f'Model parameters count: {count_parameters(model)}')
    #数据加载
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=True,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        drop_last=False,
        pin_memory=True
    )
    print(f'train batch num: {len(train_loader)}')
    print(f'val batch num: {len(val_loader)}')
            
    # Optimizer
    optimizer = getattr(torch.optim, optim)(model.parameters(), lr=LR)
    # Scheduler
    num_warmup_steps = int(0.1 * EPOCHS * len(train_loader))
    num_training_steps = int(EPOCHS * len(train_loader))
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps, num_training_steps
    )
    print(f'optim: {optim}, lr: {LR}, warmup_steps: {num_warmup_steps}')
    
    
    #train
    bst_epoch={_:0 for _ in range(8)}
    score_best={_:0 for _ in range(8)}
    first_epoch_eval=0
    for epoch in range(EPOCHS):
        
        model.train()
        model.zero_grad()
        start_time = time.time()

        avg_loss = 0
        for data in train_loader:
            pred = model(data[input].to(device))
#             print(pred.shape,data['y'].shape)

            loss = loss_fct(
                pred,
                data[y].to(device)
            ).mean()

            loss.backward()
            avg_loss += loss.item() / len(train_loader)

            optimizer.step()
            scheduler.step()

            optimizer.zero_grad(set_to_none=True)

        #VAL
        model.eval()
        mae, avg_val_loss = 0, 0
        preds = []

        with torch.no_grad():
            for data in val_loader:
                pred = model(data[input].to(device))

                loss = loss_fct(
                    pred,
                    data[y].to(device)
                ).mean()

                avg_val_loss += loss.item() / len(val_loader)

                preds.append(pred.detach().cpu().numpy())

        preds = np.concatenate(preds, 0)
        if y=='y':
            mae = compute_metric(preds,val_dataset.df[target_cols].values).mean()
        else:
            mae = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
        
        elapsed_time = time.time() - start_time
        if (epoch + 1) % verbose == 0:
            elapsed_time = elapsed_time * verbose
            lr = scheduler.get_last_lr()[0]
    #         lr=LR
            print(
                f"Epoch {epoch + 1:02d}/{ EPOCHS:02d} \t lr={lr:.1e}\t t={elapsed_time:.0f}s \t"
                f"loss={avg_loss:.4f}",
                end="\t",
            )

            if (epoch + 1 >= first_epoch_eval) or (epoch + 1 == EPOCHS):
                print(f"val_loss={avg_val_loss:.4f}\tcorr={mae:.4f}")
            else:
                print("")
                
        #保存最优模型
        score1 = compute_metric(preds,val_dataset.df[target_cols].values)
        for i in range(8):
            if score1[i] > score_best[i]:
                bst = save_model_weights(model, modelpath, f'{modelname}_target{i}_{fold_}.pt')
                score_best[i] = score1[i]
                bst_epoch[i] = epoch
                
    bst_list = [score_best[i] for i in range(8)]
    print(bst_list,'\n\n')
    del (val_loader, train_loader, loss, data, pred)
    gc.collect()
    torch.cuda.empty_cache()
    
    return bst_list


In [18]:
# K折训练，同时
seed = 666
BATCH_SIZE = 400
BATCH_SIZE_TEST = 128
EPOCHS = 300
LR = 0.001
optim = "Adam"
device = "cuda" if torch.cuda.is_available() else "cpu"
modelpath = r'./model/'

loss_fct = CVPRLoss_tanh()
k=5
scoref = []
skf = KFold(n_splits=k, shuffle=False)
for index, (train_index, test_index) in enumerate(skf.split(train_df)):   
    print(f'FOLD {index}')
    train0 = train_df.iloc[train_index]
    val0 = train_df.iloc[test_index]   
    train_dataset = MyDataset(train0, use_cols, target_cols)
    val_dataset = MyDataset(val0, use_cols, target_cols)
    print(f'train size: {len(train0)}, val size: {len(val0)}')

    modelname = f'lstm2y_catall_tanh_sig'
    seed_everything(seed)
    model = CVPRModel(input_dim=1,
                    num_classes=8,
                    bi=True,
                    time_step=37
                   ).to(device)

    _ = train_sig(model, 
                train_dataset, 
                val_dataset, 
                verbose=10, 
                fold_=index,
                modelname=modelname,
                modelpath=modelpath,
                input='input',
                y='y',
                debug=False
                 )
    scoref.append(_)
scoreff = scoref
print(np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)])

FOLD 0
train size: 400, val size: 100
Model parameters count: 102162440
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.001, warmup_steps: 30
Epoch 10/300 	 lr=3.3e-04	 t=4s 	loss=0.9255	val_loss=0.9016	corr=0.3229
Epoch 20/300 	 lr=6.7e-04	 t=4s 	loss=0.5186	val_loss=0.5736	corr=0.4322
Epoch 30/300 	 lr=1.0e-03	 t=5s 	loss=0.4077	val_loss=0.4369	corr=0.5663
Epoch 40/300 	 lr=9.6e-04	 t=5s 	loss=0.3265	val_loss=0.3514	corr=0.6513
Epoch 50/300 	 lr=9.3e-04	 t=5s 	loss=0.2783	val_loss=0.3041	corr=0.6973
Epoch 60/300 	 lr=8.9e-04	 t=5s 	loss=0.2267	val_loss=0.2442	corr=0.7606
Epoch 70/300 	 lr=8.5e-04	 t=5s 	loss=0.1995	val_loss=0.2180	corr=0.7837
Epoch 80/300 	 lr=8.1e-04	 t=4s 	loss=0.1876	val_loss=0.2166	corr=0.7861
Epoch 90/300 	 lr=7.8e-04	 t=5s 	loss=0.1837	val_loss=0.2138	corr=0.7883
Epoch 100/300 	 lr=7.4e-04	 t=4s 	loss=0.1815	val_loss=0.2133	corr=0.7878
Epoch 110/300 	 lr=7.0e-04	 t=4s 	loss=0.1793	val_loss=0.2117	corr=0.7897
Epoch 120/300 	 lr=6.7e-04	 t=4s 	loss=0.1769	

In [19]:
#加载test
with open('./data/CVPR_2022_NAS_Track2_test.json', 'r') as f:
    test_data = json.load(f)

In [20]:
test_df = get_df(test_data)
get_step(test_df)

In [21]:
def pred_cuda(test_df, model):
    #获得预测
    test_dataset = MyDataset(test_df, use_cols, target_cols, 0)
    test_loader = DataLoader(
        test_dataset,
        batch_size=1024*8,
        shuffle=False,
        drop_last=False,
        pin_memory=True
    )

    preds = []
    model.eval()
    with torch.no_grad():
        for data in tqdm(test_loader):
            pred = model(data['input'].to(device))
            preds.append(pred.detach().cpu().numpy())
    preds = np.concatenate(preds, 0)
    print(preds.shape)

    del test_dataset, test_loader, model;
    gc.collect()
    torch.cuda.empty_cache()
    return preds
        
def predict_sig(test_df, k=5, modelname=''):
    for target in range(8):        
        cols = []
        for fold_ in range(k):
            modelname1 = f'{modelname}_target{target}_{fold_}.pt'
            print(f'Model {modelname1}')
            model = CVPRModel(input_dim=1,
                        num_classes=8,
                        bi=True,
                        time_step=37
                       ).to(device)
            model.load_state_dict(torch.load(modelpath+modelname1))

            pred_ = pred_cuda(test_df, model)[:, target]
            tmp_c = f'{target_cols[target]}_{fold_}'
            test_df[tmp_c] = pred_
            cols.append(tmp_c)
            print(f'Done {tmp_c}')
            
        print(cols)
        test_df[cols] = test_df[cols].rank()
        test_df[target_cols[target]] = test_df[cols].mean(axis=1).rank()


In [22]:
k, modelname

(5, 'lstm2y_catall_tanh_sig')

In [23]:
predict_sig(test_df, k=5, modelname='lstm2y_catall_tanh_sig')

Model lstm2y_catall_tanh_sig_target0_0.pt


C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/13 [00:00<?, ?it/s]

(99500, 8)
Done cplfw_rank_0
Model lstm2y_catall_tanh_sig_target0_1.pt


  0%|          | 0/13 [00:00<?, ?it/s]

...

(99500, 8)
Done sop_rank_2
Model lstm2y_catall_tanh_sig_target7_3.pt


  0%|          | 0/13 [00:00<?, ?it/s]

(99500, 8)
Done sop_rank_3
Model lstm2y_catall_tanh_sig_target7_4.pt


  0%|          | 0/13 [00:00<?, ?it/s]

(99500, 8)
Done sop_rank_4
['sop_rank_0', 'sop_rank_1', 'sop_rank_2', 'sop_rank_3', 'sop_rank_4']


In [24]:
test_df[target_cols] = test_df[target_cols].astype(int)-1

In [25]:
test_df[target_cols].describe()

,cplfw_rank,market1501_rank,dukemtmc_rank,msmt17_rank,veri_rank,vehicleid_rank,veriwild_rank,sop_rank
count,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000
mean,49749.419859,49749.430492,49749.430683,49749.425065,49749.427608,49749.425377,49749.431628,49749.432332
std,28723.321364,28723.319863,28723.320072,28723.320062,28723.321384,28723.320223,28723.319309,28723.322151
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000
50%,49749.500000,49749.000000,49749.500000,49749.000000,49749.000000,49749.500000,49749.000000,49749.500000
75%,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000
max,99499.000000,99499.000000,99498.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000


In [26]:
test_df[target_cols].nunique()

cplfw_rank         90389
market1501_rank    91825
dukemtmc_rank      91813
msmt17_rank        91106
veri_rank          91430
vehicleid_rank     91136
veriwild_rank      91772
sop_rank           91921
dtype: int64

In [27]:
modelname

'lstm2y_catall_tanh_sig'

In [28]:
# to_sub
def to_sub(test_df, test_data, name='CVPR_2022_lgb_score'):
    for i in tqdm(test_df[['id']+target_cols].values):
        id_ = i[0]
        for k,v in enumerate(target_cols):
            k += 1
            test_data[id_][v] = i[k]
            
    with open(f'./sub/{name}.json', 'w') as f:
        json.dump(test_data, f)

In [29]:
to_sub(test_df, test_data, name='CVPR_2022_lstm2y_catall_tanh_sig')

C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/99500 [00:00<?, ?it/s]